In [61]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [80]:
data = pd.read_csv('emoji_data.csv', header = None)
data.head()


,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [63]:
data=data.dropna()

In [64]:
emoji_dict = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
    return emoji.emojize(emoji_dict[label])

In [65]:
X = data[0].values
Y = data[1].values
X=np.delete(X,29)
Y=np.delete(Y,29)

# Embeddings

In [66]:
file = open('glove.6B.100d.txt', 'r', encoding = 'utf8')
content = file.readlines()
file.close()

# content

In [67]:
embeddings = {}

for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:], dtype = float)

In [68]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    return maxlen



In [69]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word2index = tokenizer.word_index

In [70]:
Xtokens = tokenizer.texts_to_sequences(X)


In [71]:
maxlen = get_maxlen(Xtokens)
print(maxlen)

10


In [72]:
Xtrain = pad_sequences(Xtokens, maxlen = maxlen,  padding = 'post', truncating = 'post')

In [73]:
Ytrain = to_categorical(Y)

# Model

In [74]:
embed_size = 100
embedding_matrix = np.zeros((len(word2index)+1, embed_size))

for word, i in word2index.items():
    embed_vector = embeddings[word]
    embedding_matrix[i] = embed_vector

In [75]:
embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.046539,  0.61966 ,  0.56647 , ..., -0.37616 , -0.032502,
         0.8062  ],
       [-0.49886 ,  0.76602 ,  0.89751 , ..., -0.41179 ,  0.40539 ,
         0.78504 ],
       ...,
       [-0.46263 ,  0.069864,  0.69095 , ..., -0.29174 ,  0.32041 ,
         0.21202 ],
       [ 0.073242,  0.11134 ,  0.62281 , ...,  0.53417 , -0.1646  ,
        -0.27516 ],
       [ 0.29019 ,  0.80497 ,  0.31187 , ..., -0.33603 ,  0.45998 ,
        -0.11278 ]])

In [76]:
model = Sequential([
    Embedding(input_dim = len(word2index) + 1,
              output_dim = embed_size,
              input_length = maxlen,
              weights = [embedding_matrix],
              trainable = False
             ),
    
    LSTM(units = 16, return_sequences = True),
    LSTM(units = 4),
    Dense(5, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [77]:
model.fit(Xtrain, Ytrain, epochs = 100)

Epoch 1/100
6/6 [==============================] - 5s 7ms/step - loss: 1.5983 - accuracy: 0.2747
Epoch 2/100
6/6 [==============================] - 0s 6ms/step - loss: 1.5653 - accuracy: 0.3077
Epoch 3/100
6/6 [==============================] - 0s 6ms/step - loss: 1.5486 - accuracy: 0.2637
Epoch 4/100
6/6 [==============================] - 0s 6ms/step - loss: 1.5355 - accuracy: 0.2857
Epoch 5/100
6/6 [==============================] - 0s 6ms/step - loss: 1.5219 - accuracy: 0.3022
Epoch 6/100
6/6 [==============================] - 0s 5ms/step - loss: 1.5032 - accuracy: 0.3132
Epoch 7/100
6/6 [==============================] - 0s 5ms/step - loss: 1.4832 - accuracy: 0.3297
Epoch 8/100
6/6 [==============================] - 0s 5ms/step - loss: 1.4612 - accuracy: 0.3681
Epoch 9/100
6/6 [==============================] - 0s 5ms/step - loss: 1.4337 - accuracy: 0.4011
Epoch 10/100
6/6 [==============================] - 0s 5ms/step - loss: 1.4050 - accuracy: 0.4066
Epoch 11/100
6/6 [===========

6/6 [==============================] - 0s 5ms/step - loss: 0.2457 - accuracy: 0.9835
Epoch 85/100
6/6 [==============================] - 0s 6ms/step - loss: 0.2432 - accuracy: 0.9835
Epoch 86/100
6/6 [==============================] - 0s 6ms/step - loss: 0.2408 - accuracy: 0.9835
Epoch 87/100
6/6 [==============================] - 0s 6ms/step - loss: 0.2385 - accuracy: 0.9835
Epoch 88/100
6/6 [==============================] - 0s 5ms/step - loss: 0.2360 - accuracy: 0.9835
Epoch 89/100
6/6 [==============================] - 0s 5ms/step - loss: 0.2339 - accuracy: 0.9835
Epoch 90/100
6/6 [==============================] - 0s 5ms/step - loss: 0.2316 - accuracy: 0.9835
Epoch 91/100
6/6 [==============================] - 0s 5ms/step - loss: 0.2294 - accuracy: 0.9835
Epoch 92/100
6/6 [==============================] - 0s 5ms/step - loss: 0.2273 - accuracy: 0.9835
Epoch 93/100
6/6 [==============================] - 0s 5ms/step - loss: 0.2253 - accuracy: 0.9835
Epoch 94/100
6/6 [===============

In [78]:
test = ["I feel good", "I feel very bad", "lets eat dinner"]

test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')

y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis = 1)

for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

I feel good 😃
I feel very bad 😞
lets eat dinner 🍽️


In [116]:
def check(*args):
    
    test = [str(e1.get())]

    test_seq = tokenizer.texts_to_sequences(test)
    Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')

    y_pred = model.predict(Xtest)
    y_pred = np.argmax(y_pred, axis = 1)

    for i in range(len(test)):
        outtext=test[i]+label_to_emoji(y_pred[i])
    messagebox.showinfo("Alert",outtext)

In [127]:
root=tkinter.Tk()
root.geometry("200x200")
e1=tkinter.Entry(root)
e1.grid(row=1,column=3)
b1=tkinter.Button(root,command=check,text="press")
b1.grid(row=2,column=3)
lab1=tkinter.Label(root)
lab1.grid(row=3,column=1)

root.mainloop()

In [91]:
import tkinter
from tkinter import messagebox